# Model Training

In this exercise we will train a model on the ferries and weather sets from earlier today. We'll be using a mix of `polars` and `scikit-learn` for some feature engineering and preprocessing of the data. The model will be deployed to and served from [Posit Connect](https://pub.ferryland.posit.team/) using [`pins`](https://rstudio.github.io/pins-python/) and [`vetiver`](https://rstudio.github.io/vetiver-python/stable/). For this section, we'll be using a [random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#) model to predict the delay in ferry departures.

## Preliminaries

First we'll load our environment variables from `.env` file and get our Connect username using the [Posit SDK for Python](https://github.com/posit-dev/posit-sdk-py).

In [15]:
# Load environment variables from .env.
import os
from pathlib import Path
from dotenv import load_dotenv

if Path(".env").exists():
    load_dotenv()

In [16]:
# Get Connect username.
from posit.connect import Client

connect_url = os.environ["CONNECT_SERVER"]
connect_api_key = os.environ["CONNECT_API_KEY"]

with Client(url=connect_url, api_key=connect_api_key) as client:
    username = client.me.username
print(username)

brooklynbagel


## Task 0 - Reading the data

First we'll read the vessel history, vessel verbose and terminal weather data was that written into the database in the previous sections.

In [17]:
import polars as pl

db_uri = os.environ["DATABASE_URI_PYTHON"]

vessel_history = pl.read_database_uri(
    query=f"SELECT * FROM vessel_history_clean;", uri=db_uri, engine="adbc"
)
vessel_verbose = pl.read_database_uri(
    query=f"SELECT * FROM vessel_verbose_clean;", uri=db_uri, engine="adbc"
)
weather = pl.read_database_uri(
    query=f"SELECT * FROM terminal_weather_clean;", uri=db_uri, engine="adbc"
)

Let's take a look at some of the features available that we can train on.

In [18]:
vessel_history.glimpse()

Rows: 536342
Columns: 7
$ Vessel                        <str> 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet'
$ Departing                     <str> 'vashon island', 'fauntleroy', 'vashon island', 'fauntleroy', 'vashon island', 'southworth', 'vashon island', 'fauntleroy', 'southworth', 'vashon island'
$ Arriving                      <str> 'fauntleroy', 'vashon island', 'fauntleroy', 'vashon island', 'southworth', 'vashon island', 'fauntleroy', 'southworth', 'vashon island', 'fauntleroy'
$ ScheduledDepart <datetime[μs, UTC]> 2020-01-01 13:40:00+00:00, 2020-01-01 14:10:00+00:00, 2020-01-01 14:35:00+00:00, 2020-01-01 15:05:00+00:00, 2020-01-01 15:30:00+00:00, 2020-01-01 15:55:00+00:00, 2020-01-01 16:15:00+00:00, 2020-01-01 16:45:00+00:00, 2020-01-01 17:20:00+00:00, 2020-01-01 17:40:00+00:00
$ ActualDepart    <datetime[μs, UTC]> 2020-01-01 13:41:12+00:00, 2020-01-01 14:11:37+00:00, 2020-01-01 14:37:37+00:00, 20

In [19]:
vessel_verbose.glimpse()

Rows: 21
Columns: 44
$ VesselID                           <i64> 1, 2, 65, 74, 15, 17, 52, 18, 19, 25
$ VesselSubjectID                    <i64> 1, 2, 428, 487, 15, 17, 52, 18, 19, 25
$ VesselName                         <str> 'cathlamet', 'chelan', 'chetzemoka', 'chimacum', 'issaquah', 'kaleetan', 'kennewick', 'kitsap', 'kittitas', 'puyallup'
$ VesselAbbrev                       <str> 'cat', 'che', 'chz', 'chm', 'iss', 'kal', 'ken', 'kis', 'kit', 'puy'
$ ClassID                            <i64> 10, 10, 162, 100, 10, 50, 162, 10, 10, 90
$ ClassSubjectID                     <i64> 310, 310, 427, 319, 310, 314, 427, 310, 310, 318
$ ClassName                          <str> 'issaquah 130', 'issaquah 130', 'kwa-di tabil', 'olympic', 'issaquah 130', 'super', 'kwa-di tabil', 'issaquah 130', 'issaquah 130', 'jumbo mark ii'
$ SortSeq                            <i64> 40, 40, 75, 35, 40, 30, 75, 40, 40, 10
$ DrawingImg                         <str> 'https://www.wsdot.wa.gov/ferries/images/pages/boa

In [20]:
weather.glimpse()

Rows: 800160
Columns: 16
$ latitude                            <f64> 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298
$ longitude                           <f64> -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264
$ generationtime_ms                   <f64> 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453, 0.2510547637939453
$ utc_offset_seconds                  <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ timezone                            <str> 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt'
$ timezone_abbreviation               <str> 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt', 'gmt'
$ elevation                           <f64> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
$ time        

## Task 1 - Feature Engineering

We're interested in modeling what the departure delay for a given ferry trip might be. While there is no delay column, we can derive it using the `ActualDepart` and `ScheduleDepart` columns. 

We're also interested in separating out the `Date` column into year, month, weekday and hour components in order for our model to pick up on these features.

In [21]:
ferry_trips = vessel_history.select(
    pl.col("Vessel", "Departing", "Arriving"),
    (pl.col("ActualDepart") - pl.col("ScheduledDepart"))
    .dt.total_seconds()
    .alias("Delay"),
    pl.col("Date"),
    pl.col("Date").dt.year().alias("Year"),
    pl.col("Date").dt.month().alias("Month"),
    pl.col("Date").dt.weekday().alias("Weekday"),
    pl.col("Date").dt.hour().alias("Hour"),
)
ferry_trips

Vessel,Departing,Arriving,Delay,Date,Year,Month,Weekday,Hour
str,str,str,i64,"datetime[μs, UTC]",i32,i8,i8,i8
"""cathlamet""","""vashon island""","""fauntleroy""",72,2020-01-01 13:40:00 UTC,2020,1,3,13
"""cathlamet""","""fauntleroy""","""vashon island""",97,2020-01-01 14:10:00 UTC,2020,1,3,14
"""cathlamet""","""vashon island""","""fauntleroy""",157,2020-01-01 14:35:00 UTC,2020,1,3,14
"""cathlamet""","""fauntleroy""","""vashon island""",131,2020-01-01 15:05:00 UTC,2020,1,3,15
"""cathlamet""","""vashon island""","""southworth""",61,2020-01-01 15:30:00 UTC,2020,1,3,15
…,…,…,…,…,…,…,…,…
"""yakima""","""shaw island""","""anacortes""",4949,2024-07-25 00:30:00 UTC,2024,7,4,0
"""yakima""","""orcas island""","""shaw island""",872,2024-07-25 01:20:00 UTC,2024,7,4,1
"""yakima""","""anacortes""","""lopez island""",638,2024-07-25 04:00:00 UTC,2024,7,4,4


A quick look at the `Delay` data shows that there's significant skew and even some negative delays.

In [22]:
ferry_trips.plot.hist("Delay", bin_range=(-1800, 7200), bins=30)

:Histogram   [Delay]   (Delay_count)

 For the purposes of making it easier to model we'll assume delays can only be non-negative and log them in order to get a nicer distribution for regression. 

In [23]:
ferry_trips.select(
    pl.col("Delay")
    .map_elements(lambda x: max(x, 1), return_dtype=pl.Float64)
    .log()
    .alias("LogDelay")
).plot.hist("LogDelay")

:Histogram   [LogDelay]   (LogDelay_count)

In [24]:
ferry_trips = ferry_trips.select(
    pl.exclude("Delay"),
    pl.col("Delay")
    .map_elements(lambda x: max(x, 1), return_dtype=pl.Float64)
    .log()
    .alias("LogDelay"),
)
ferry_trips

Vessel,Departing,Arriving,Date,Year,Month,Weekday,Hour,LogDelay
str,str,str,"datetime[μs, UTC]",i32,i8,i8,i8,f64
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 13:40:00 UTC,2020,1,3,13,4.276666
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 14:10:00 UTC,2020,1,3,14,4.574711
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 14:35:00 UTC,2020,1,3,14,5.056246
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 15:05:00 UTC,2020,1,3,15,4.875197
"""cathlamet""","""vashon island""","""southworth""",2020-01-01 15:30:00 UTC,2020,1,3,15,4.110874
…,…,…,…,…,…,…,…,…
"""yakima""","""shaw island""","""anacortes""",2024-07-25 00:30:00 UTC,2024,7,4,0,8.506941
"""yakima""","""orcas island""","""shaw island""",2024-07-25 01:20:00 UTC,2024,7,4,1,6.770789
"""yakima""","""anacortes""","""lopez island""",2024-07-25 04:00:00 UTC,2024,7,4,4,6.458338


Now we'll want to join the ferry data describing the vessels the trips were taken in. First we're selecting a subset of the columns and extracting the year from the `YearBuilt` and `YearRebuilt` columns.

In [25]:
ferry_info = vessel_verbose.select(
    pl.col("VesselName").str.to_lowercase(),
    pl.col("ClassName"),
    pl.col(
        "SpeedInKnots",
        "EngineCount",
        "Horsepower",
        "MaxPassengerCount",
        "PassengerOnly",
        "FastFerry",
        "PropulsionInfo",
    ),
    pl.col("YearBuilt", "YearRebuilt").dt.year(),
)

ferry_trips = ferry_trips.join(
    ferry_info, left_on="Vessel", right_on="VesselName", how="left", coalesce=True
)

The weather data has a granularity of one hour, so in order to join this with the `ferry_trips` data we're going to round the timestamp associated with the trip to the nearest hour. We're going to join in the weather data twice for both the departing terminal and arriving terminal. Finally, there a number of columns associated with the weather data that are not needed and will be dropped.

In [26]:
import polars.selectors as cs

ferry_trips = (
    ferry_trips.with_columns(pl.col("Date").dt.round("1h").alias("time"))
    .join(
        weather.rename(lambda col_name: f"departing_{col_name}"),
        how="left",
        left_on=["Departing", "time"],
        right_on=["departing_terminal_name", "departing_time"],
        coalesce=True,
    )
    .join(
        weather.rename(lambda col_name: f"arriving_{col_name}"),
        how="left",
        left_on=["Arriving", "time"],
        right_on=["arriving_terminal_name", "arriving_time"],
        coalesce=True,
    )
    .select(
        ~cs.ends_with(
            "latitude",
            "longitude",
            "generationtime_ms",
            "utc_offset_seconds",
            "timezone",
            "timezone_abbreviation",
            "elevation",
            "hourly_units",
        ),
    )
    .select(pl.exclude("time"))
)
ferry_trips

Vessel,Departing,Arriving,Date,Year,Month,Weekday,Hour,LogDelay,ClassName,SpeedInKnots,EngineCount,Horsepower,MaxPassengerCount,PassengerOnly,FastFerry,PropulsionInfo,YearBuilt,YearRebuilt,departing_weather_code,departing_temperature_2m,departing_precipitation,departing_cloud_cover,departing_wind_speed_10m,departing_wind_direction_10m,departing_wind_gusts_10m,arriving_weather_code,arriving_temperature_2m,arriving_precipitation,arriving_cloud_cover,arriving_wind_speed_10m,arriving_wind_direction_10m,arriving_wind_gusts_10m
str,str,str,"datetime[μs, UTC]",i32,i8,i8,i8,f64,str,i64,i64,i64,i64,bool,bool,str,i32,i32,i64,f64,f64,i64,f64,i64,f64,i64,f64,f64,i64,f64,i64,f64
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 13:40:00 UTC,2020,1,3,13,4.276666,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,3,9.2,0.0,87,26.2,226,53.3,3,9.1,0.0,92,31.6,222,55.1
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 14:10:00 UTC,2020,1,3,14,4.574711,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,3,9.1,0.0,92,31.6,222,55.1,3,9.2,0.0,87,26.2,226,53.3
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 14:35:00 UTC,2020,1,3,14,5.056246,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,51,8.9,0.1,60,26.7,212,56.5,2,8.7,0.0,77,34.6,211,63.4
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 15:05:00 UTC,2020,1,3,15,4.875197,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,2,8.7,0.0,77,34.6,211,63.4,51,8.9,0.1,60,26.7,212,56.5
"""cathlamet""","""vashon island""","""southworth""",2020-01-01 15:30:00 UTC,2020,1,3,15,4.110874,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,1,8.8,0.0,20,18.8,238,54.4,1,8.7,0.0,20,18.8,238,54.4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""yakima""","""shaw island""","""anacortes""",2024-07-25 00:30:00 UTC,2024,7,4,0,8.506941,"""super""",17,4,8000,2000,false,false,"""diesel-electric (dc)""",1967,2000,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""yakima""","""orcas island""","""shaw island""",2024-07-25 01:20:00 UTC,2024,7,4,1,6.770789,"""super""",17,4,8000,2000,false,false,"""diesel-electric (dc)""",1967,2000,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""yakima""","""anacortes""","""lopez island""",2024-07-25 04:00:00 UTC,2024,7,4,4,6.458338,"""super""",17,4,8000,2000,false,false,"""diesel-electric (dc)""",1967,2000,null,null,null,null,null,null,null,null,null,null,null,null,null,null


## Task 2 - Preprocessing and Modeling

### 🔄 Task

Define a `scikit-learn` pipeline that

- Transform the data for the model to ingest
- Trains a random forest model to predict the logged departure delay

### 🧑‍💻 Code

First we separate the columns in numeric features and categorical features. Our random forest model requires the categorical features be one-hot encoded while our numeric features can be left as-is.

In [27]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = [
    "Month",
    "Weekday",
    "Hour",
    "SpeedInKnots",
    "EngineCount",
    "Horsepower",
    "MaxPassengerCount",
    # "PassengerOnly",
    # "FastFerry",
    "YearBuilt",
    "YearRebuilt",
    "departing_temperature_2m",
    # "departing_precipitation",
    "departing_cloud_cover",
    "departing_wind_speed_10m",
    "departing_wind_direction_10m",
    "departing_wind_gusts_10m",
    "arriving_temperature_2m",
    # "arriving_precipitation",
    "arriving_cloud_cover",
    "arriving_wind_speed_10m",
    "arriving_wind_direction_10m",
    "arriving_wind_gusts_10m",
]
categorical_features = [
    "Departing",
    "Arriving",
    "ClassName",
    "PropulsionInfo",
    "departing_weather_code",
    "arriving_weather_code",
]


preprocessor = ColumnTransformer(
    [
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(), categorical_features),
    ]
)

Here we define our random forest model.

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(verbose=2, random_state=2, n_jobs=-1)

Now our preprocessor and random forest model are joined together into a single pipeline. This makes using the model easier as we won't have to feed in pre-processed data - the pipeline will take of that step for us during inference.

In [29]:
from sklearn.pipeline import Pipeline

model = Pipeline([("preprocess", preprocessor), ("random-forest", rf)])

Now we're filtering the data, keeping only the data from the past year, and then splitting into a train and test set.

In [30]:
import datetime


from sklearn.model_selection import train_test_split

ferry_trips_filtered = ferry_trips.drop_nulls().filter(
    pl.col("Date").dt.date() >= (datetime.date.today() - datetime.timedelta(weeks=53))
)

X = ferry_trips_filtered.drop("Vessel", "Date", "Year", "LogDelay")

# TODO: review issues with prototype data
X = X.drop(
    "PassengerOnly",
    "FastFerry",
    "arriving_precipitation",
    "departing_precipitation",
)
y = ferry_trips_filtered["LogDelay"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
print(f"Nrows training data: {X_train.shape[0]}")
print(f"Nrows testing data:  {X_test.shape[0]}")

Nrows training data: 46485
Nrows testing data:  11622


In order to use the test later for our model card (to be discussed) the test data will be saved to the database.

In [31]:
X_test.with_columns(y_test).write_database(
    table_name="test_data", connection=db_uri, engine="adbc", if_table_exists="replace"
)

11622

Finally, we train the model and compute the r-squared using the test data set.

In [32]:
%%time
model.fit(X_train.to_pandas(), y_train)
model.score(X_test, y_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.0min


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  8.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s


CPU times: user 29min 18s, sys: 19.7 s, total: 29min 37s
Wall time: 8min 38s


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.3s finished


0.32387919683985134

## Task 3 - Deploying

### 🔄 Task

- Deploy the model using `vetiver` and `pins` onto Posit Connect
- Deploy an API around the model onto Posit

### 🧑‍💻 Code

In [33]:
from vetiver import VetiverModel

v = VetiverModel(
    model, model_name=f"{username}/ferry_delay", prototype_data=X.to_pandas()
)

In [34]:
import pins
import vetiver

model_board = pins.board_connect(
    server_url=connect_url, api_key=connect_api_key, allow_pickle_read=True
)
vetiver.vetiver_pin_write(model_board, model=v)

Model Cards provide a framework for transparent, responsible reporting. 
 Use the vetiver `.qmd` Quarto template as a place to start, 
 with vetiver.model_card()
Writing pin:
Name: 'brooklynbagel/ferry_delay'
Version: 20240731T160515Z-24e71


In [35]:
%%time
from rsconnect.api import RSConnectServer

connect_server = RSConnectServer(url=connect_url, api_key=connect_api_key)
vetiver.deploy_rsconnect(
    connect_server=connect_server,
    board=model_board,
    pin_name=f"{username}/ferry_delay",
)

             Consider creating a requirements.txt file instead.


/home/brooklynbagel/ds-workflows-python/materials/03-model-training/.venv/lib/python3.12/site-packages/vetiver/write_fastapi.py:21: UserWarning: Pinned vetiver model has no active version and no datetime on versions,
              Do you need to check your pinned model?
              Using version 121
  warnings.warn(
Validating server... 	[OK]
Validating app mode... 	[OK]
Making bundle ... 	[OK]
Deploying bundle ... 	[OK]
Saving deployed information... 	[OK]
Building FastAPI application...
Bundle created with Python version 3.12.4 is compatible with environment Kubernetes::docker.io/brooklynbagel/conf2024-ds-workflows:r4.4.0-py3.12.3-quarto1.4.557-ubuntu22.04 with Python version 3.12.3 from /opt/python/3.12.3/bin/python 
Bundle requested Python version 3.12.4; using /opt/python/3.12.3/bin/python from Kubernetes::docker.io/brooklynbagel/conf2024-ds-workflows:r4.4.0-py3.12.3-quarto1.4.557-ubuntu22.04 which has version 3.12.3
Determining session server location ...
2024/07/31 16:07:31.71

CPU times: user 167 ms, sys: 29.3 ms, total: 196 ms
Wall time: 17.9 s


## Task 4 - Model Card

### 🔄 Task

- Use a model card to describe various metrics for how the model performs
- Deploy the card to Connect

### 🧑‍💻 Code

In [ ]:
# vetiver.templates.model_card()